In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
import cv2
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Coding a simple model

In [2]:
class MLP:
    def __init__(self, neurons_in_layer, learning_rate):
        self.neurons_in_layer = neurons_in_layer   # 1 input layer, 2 hidden, 1 output layer so a 1x4 array
        self.learning_rate = learning_rate
        
        self.layers = len(neurons_in_layer) # Excluding final layer

        # Initializing weights & biases
        self.weights = []
        self.biases = []
        

        for i in range(self.layers - 1):
            # In the matrices rows correspond to neurons in the current layer and columns to neurons in the next layer
            self.weights.append(np.random.uniform(-1,1,size=[neurons_in_layer[i],neurons_in_layer[i+1]]))
            self.biases.append(np.zeros((1, neurons_in_layer[i+1])))

        print(len(self.weights[0]))
        print(len(self.weights[1]))
        print(len(self.weights[2]))    
    def sigmoid(self, x):
        return 1./(1.+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        sigmoid_x = self.sigmoid(x)
        return sigmoid_x * (1 - sigmoid_x)
    
    def ReLU(self, x):
        return np.maximum(0, x)
    
    def ReLU_derivative(self, x):
        return (x>0)*1
    
    # To have probabilities (all sum up to 1) in the output layer
    def softmax(self, x):
        print("x:", x)
        exp_x = np.exp(x)
        print("exp_x:", exp_x)
        prob = exp_x / exp_x.sum(axis=0, keepdims=True)
        print("prob:", prob)
        return prob
    
    def feed_forward(self, batch):
        self.layer_output = [batch]

        # Passing through each layer except the last one
        for i in range(self.layers - 1):
            output = self.layer_output[-1].dot(self.weights[i]) + self.biases[i]   # weight * x + bias
            self.layer_output.append(self.ReLU(output))

        # Using softmax in the output layer
        self.layer_output[-1] = self.softmax(self.layer_output[-1])
        
        print(self.layer_output[-1])
        return self.layer_output[-1]
        #final_x = np.dot(self.layer_output[-1], self.weights[-1]) + self.biases[-1]
        #self.x_values.append(final_x)
        #self.layer_output.append(self.softmax(final_x))

    def backpropagation(self, y, X_size):
        output_error = self.layer_output[-1] - y  # cross- entropy loss
        delta = output_error * self.ReLU_derivative(self.layer_output[-1])

        # Backpropagation through each layer backwards (duh)
        for i in range(1, self.layers - 1):
            self.weights[-i] -= self.learning_rate * (self.layer_output[-i-1].T.dot(delta))
            self.biases[-i] -= self.learning_rate * np.mean(delta, axis=0, keepdims=True)
            delta = self.ReLU_derivative(self.layer_output[-i-1]) * (delta.dot(self.weights[-i].T))
            # delta = deltas[-1].dot(self.weights[i+1].T) * self.ReLU_derivative(self.layer_output[i+1])
            # deltas.append(delta)

        # # Reversing deltas to match layer order
        # deltas.reverse()

        # # Updating weights and biases
        # for i in range(self.layers - 1):
        #     self.weights[i] -= self.learning_rate * self.layer_output[i].T.dot(deltas[i])
        #     self.biases[i] -= self.learning_rate * np.sum(deltas[i], axis=0, keepdims=True)

    def train(self, X, y, epochs = 1000):
        for epoch in range(epochs):
            output = self.feed_forward(X)
            self.backpropagation(y, len(X))
            #print(output)
            if epoch % 1 == 0:
                loss = -np.sum(y * np.log(output + 1e-9)) / X.shape[0]
                print(f'Epoch {epoch}, Loss: {loss}')

    def predict(self, X):
        output = self.feed_forward(X)
        print("Output:", output)
        print("Label:", np.argmax(output, axis=1))
        return np.argmax(output, axis=1)

    

In [3]:
class MLP_new:
    def __init__(self, input, hidden_layer1, hidden_layer2, output, learning_rate, sample_size):
        self.input = input
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = hidden_layer2
        self.output = output
        self.learning_rate = learning_rate
        
        self.sample_size = sample_size
        self.total_error = 0

        # Initializing weights and biases
        self.weights1 = np.random.uniform(-1, 1, size=[input, hidden_layer1])
        self.weights2 = np.random.uniform(-1, 1, size=[hidden_layer1, hidden_layer2])
        self.weights3 = np.random.uniform(-1, 1, size=[hidden_layer2, output])

        self.biases1 = np.zeros((1, hidden_layer1))
        self.biases2 = np.zeros((1, hidden_layer2))
        self.biases3 = np.zeros((1, output))

        self.final_output = []
        self.loss = []


    # Defining useful functions
    def sigmoid(self, x):
        return 1/(1+np.exp(-x * 1.0))
        
    def sigmoid_prime(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def ReLU(self, x):
        return np.maximum(0, x)
    
    def ReLU_prime(self,x):
        return (x>0)*1
    
    def softmax(self,x):
        x_shifted = x - np.max(x, axis=1, keepdims=True)
        exp = np.exp(x_shifted)
        probabilities = exp / np.sum(exp, axis=1, keepdims=True)
        return probabilities
    
    def cross_entropy_loss(self, y):
        loss = 0
        for sample in range(y.shape[0]):
            final_output_label = np.argmax(self.final_output[sample])
            y_label = np.argmax(y[sample])
            
            loss = loss + (-1 * y_label*np.log(final_output_label+ 1e-9))
        return (loss/y.shape[0])
    
    def return_loss(self):
        return self.loss

    # Forward
    def train(self, X, y, epochs):
        self.loss = np.zeros([epochs,1])
        for epoch in range(epochs):
            self.final_output = np.zeros([self.sample_size, self.output])
            for sample, inputs in enumerate(X):
                inputs = inputs.reshape(-1,1).T
                # Forward pass
                output1 = self.sigmoid(inputs.dot(self.weights1) + self.biases1)
                output2 = self.sigmoid(output1.dot(self.weights2) + self.biases2)
                output3 = self.softmax(output2.dot(self.weights3) + self.biases3)

                self.final_output[sample] = output3

                # Backward pass
                error = output3 - y[sample]
                self.total_error += error
                
                delta_out = error * self.sigmoid_prime(output3)
                self.weights3 -= self.learning_rate * (output2.T.dot(delta_out))
                self.biases3 -= self.learning_rate * delta_out
        
                delta2 = self.sigmoid_prime(output2) * (delta_out.dot(self.weights3.T))
                self.weights2 -= self.learning_rate * (output1.T.dot(delta2))
                self.biases2 -= self.learning_rate * delta2

                delta1 = self.sigmoid_prime(output1) * (delta2.dot(self.weights2.T))
                self.weights1 -= self.learning_rate * (inputs.T.dot(delta1))
                self.biases1 -= self.learning_rate * delta1

            self.total_error = self.total_error/len(y)

            self.loss[epoch] = self.cross_entropy_loss(y)
            if epoch % 10 == 0:
                print(f"Epoch: {epoch}, Loss: {self.loss[epoch]}, Final output: {self.final_output}")

    def predict(self, X):
        predictions = []
        self.final_output = np.zeros([self.sample_size, self.output])
        for sample, inputs in enumerate(X):
            # Forward pass
            inputs = inputs.reshape(-1,1).T
                # Forward pass
            output1 = self.sigmoid(inputs.dot(self.weights1) + self.biases1)
            output2 = self.sigmoid(output1.dot(self.weights2) + self.biases2)
            output3 = self.softmax(output2.dot(self.weights3) + self.biases3)

            self.final_output[sample] = output3

            predictions.append(np.argmax(self.final_output[sample]))
        return predictions

### Loading data

In [4]:
data_file = "C:/Users/afrod/Documents/Neural_Networks/MergedDataset"
classes = ["NonDemented", "VeryMildDemented", "MildDemented", "ModerateDemented"]
training_data = []


def create_training_data():
    for dementia_level in classes:
        path = os.path.join(data_file, dementia_level)
        class_num = classes.index(dementia_level)
        for img in os.listdir(path):
            # Convert to grayscale for smaller array dimensions
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            final_array = cv2.resize(img_array, (50,47))
            training_data.append([final_array, class_num])

create_training_data()

In [5]:
random.shuffle(training_data)

# Separating features and labels
# Images are also flattened to be used as input in the knn algorithm
X = np.array([features for features, _ in training_data]).reshape(-1, 50*47)
y = np.array([label for _, label in training_data])

# Rescaling
X = X/X.max()

# One-hot encoding
y_onehot = np.zeros((y.size, int(y.max()) + 1))
y_onehot[np.arange(y.size),y.astype(int)] = 1.0

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.4, random_state=42)
print(X_train.shape[0])

24230


### Training time

In [12]:
mlp = MLP_new(2350, 1000, 100, 4, 0.01, len(X_train))

In [ ]:
mlp.train(X_train, y_train, 10)

Epoch: 0, Loss: [7.01968192], Final output: [[1.14329059e-01 5.45055231e-02 8.31122229e-01 4.31884423e-05]
 [3.53365735e-02 4.47625396e-02 9.19874124e-01 2.67633677e-05]
 [3.23935367e-02 3.96081761e-02 9.27970960e-01 2.73274051e-05]
 ...
 [1.48180948e-01 4.28709168e-01 3.73400101e-01 4.97097822e-02]
 [2.08409521e-01 3.60298427e-01 4.05783202e-01 2.55088501e-02]
 [8.87139876e-02 4.15186685e-01 3.23342994e-01 1.72756333e-01]]


In [8]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print("\nMLP with layers [2350, 600, 100, 4], lr = 0.01, epochs = 100:")
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print("Accuracy:", accuracy_score(y_test_labels, predictions))
print("Classification Report:\n", classification_report(y_test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))


MLP with layers [2350, 600, 100, 4], lr = 0.01, epochs = 100:
Test Accuracy: 39.96%
Accuracy: 0.39959143246254797
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.02      0.04      5078
           1       0.32      0.42      0.36      4448
           2       0.35      0.72      0.47      3978
           3       0.83      0.60      0.70      2650

    accuracy                           0.40     16154
   macro avg       0.56      0.44      0.39     16154
weighted avg       0.55      0.40      0.34     16154

Confusion Matrix:
 [[  99 3040 1891   48]
 [  17 1881 2429  121]
 [  14  918 2879  167]
 [   0   94  960 1596]]


In [11]:
mlp.loss

array([[7.71270071],
       [6.12670058]])